From: https://github.com/Quorumetrix/Blender_scripts/blob/main/Mesh%20Decimation%20Pipeline.ipynb

In [1]:
import pandas as pd
import numpy as np
import os
import time
import h5py
import pyvista as pv

# Set up paths and parameters
obj_path = 'data/neuron_meshes_v185/'
dec_prcnt = 95
dec_path = 'data/neuron_meshes_v185/dec/'

# Load cellids into a list
cell_list = [648518346349525821] #[648518346349529031]

#[648518346349508122, 648518346349525545, 648518346349514298, 648518346342800512, 648518346349530761, 648518346349524109, 648518346349530765, 648518346349524140, 648518346349525684, 648518346349525196, 648518346349529813, 648518346349508442, 648518346349538793, 648518346349508089, 648518346349530654, 648518346341355049, 648518346349528624, 648518346349510740, 648518346349530199, 648518346349531742, 648518346349530724, 648518346342801515, 648518346349510254, 648518346349375599, 648518346342797442, 648518346341391505, 648518346341351575, 648518346349539502, 648518346349503169, 648518346349529797, 648518346342792418, 648518346349524211, 648518346341382424, 648518346342919961, 648518346349508386, 648518346349516092, 648518346349508432, 648518346349368699, 648518346341394877, 648518346349538752, 648518346349529029, 648518346349502931, 648518346349508279, 648518346349525488]
# OPCs, oligos, and microglia

#[648518346349538237, 648518346349536816, 648518346349539797, 648518346349536744, 648518346349536851, 648518346349522862, 648518346349529341, 648518346349530521, 648518346341382533, 648518346349492130, 648518346349531834, 648518346349537400, 648518346349528749, 648518346349532050, 648518346349538475, 648518346349538733, 648518346349530766, 648518346349537071, 648518346349528913, 648518346349535286, 648518346349523129, 648518346349533252, 648518346349524933, 648518346349539891, 648518346349538001, 648518346349539414, 648518346349525861, 648518346349492197, 648518346349536478, 648518346349511899, 648518346349539895, 648518346349539462, 648518346349537916, 648518346349528248, 648518346349538251, 648518346349539068, 648518346349539593, 648518346349537913, 648518346349538254, 648518346349537827, 648518346349539437, 648518346349538312, 648518346349539890, 648518346349539819, 648518346349535007, 648518346349526227, 648518346349539093, 648518346349531327, 648518346349538391, 648518346349539863, 648518346349539844, 648518346349539471, 648518346349538414, 648518346349539864, 648518346349530177, 648518346349522230, 648518346349534072, 648518346349539506, 648518346349537981, 648518346349539334, 648518346349539594, 648518346349532067, 648518346349539856, 648518346349525821, 648518346349529960, 648518346349536205, 648518346349535264, 648518346349535102, 648518346349523266, 648518346349517132, 648518346349529778, 648518346349538056, 648518346349539851, 648518346349494004, 648518346349538387, 648518346349539862, 648518346349539347, 648518346349505739, 648518346349535363, 648518346349538365, 648518346349538416, 648518346349537897, 648518346349539836, 648518346349539834, 648518346349539359, 648518346349539569, 648518346349525406, 648518346349537835, 648518346349538270, 648518346349523306, 648518346349539785, 648518346349537492, 648518346349537946, 648518346349526534, 648518346349528587, 648518346349534794, 648518346349537996, 648518346349523993, 648518346349539845, 648518346349535192, 648518346349529905, 648518346349537828, 648518346349537844, 648518346349538711, 648518346349537472, 648518346349538395, 648518346349538098, 648518346349533267, 648518346349539781, 648518346349525972, 648518346349539885, 648518346349537851, 648518346349528588, 648518346349536811, 648518346349537641, 648518346349537989, 648518346349539433, 648518346349540017, 648518346349538440, 648518346349538527, 648518346349493472, 648518346349539804, 648518346349534292, 648518346349529376, 648518346349537425, 648518346349531774, 648518346349536924, 648518346349539809, 648518346349536391, 648518346349539803, 648518346349538378, 648518346349537486, 648518346349536997, 648518346349536988, 648518346349539586, 648518346349539390, 648518346349537244, 648518346349540051, 648518346349537657, 648518346349534289, 648518346349526132, 648518346349538157, 648518346349539097, 648518346349531362, 648518346349539887, 648518346349477155, 648518346349539825, 648518346349539896, 648518346349529438, 648518346349539768, 648518346349539070, 648518346349538336, 648518346349537860, 648518346349531230, 648518346349539892, 648518346349523981, 648518346349525753, 648518346349520120, 648518346349538259, 648518346349538782, 648518346349539961, 648518346349538332, 648518346349533811, 648518346349536788, 648518346349535847, 648518346349538193, 648518346349539376, 648518346349527116, 648518346349538188, 648518346349539795, 648518346349538421, 648518346349537717, 648518346349525720, 648518346349534614, 648518346349523667, 648518346349536366, 648518346349539755, 648518346349493487, 648518346349537853, 648518346349530300, 648518346349532086, 648518346349539110, 648518346349538053, 648518346349539554, 648518346349539821, 648518346349526971, 648518346349534360, 648518346349538451, 648518346349528050, 648518346349536312, 648518346349536992, 648518346349539572, 648518346349535435, 648518346349539575, 648518346349538317, 648518346349533482, 648518346349530359, 648518346349538278, 648518346349539901, 648518346349536534, 648518346349539806, 648518346349537145, 648518346349538721, 648518346349538100, 648518346349538462, 648518346349533350, 648518346349538291, 648518346349529575, 648518346349538478, 648518346349539071, 648518346349523975, 648518346349538787, 648518346349538410, 648518346349527610, 648518346349538133, 648518346349538466, 648518346349539423, 648518346349531530, 648518346349537446, 648518346349537993, 648518346349539865, 648518346349537514, 648518346349539560, 648518346349537385, 648518346349538426, 648518346349538190, 648518346349539906, 648518346349537482, 648518346349539727, 648518346349539368, 648518346349536966, 648518346349493874, 648518346349538239, 648518346349494577, 648518346349539096, 648518346349539095, 648518346349539853, 648518346349539832, 648518346349537687, 648518346349540053, 648518346349531526, 648518346349538015, 648518346349523030, 648518346349539599, 648518346349523254, 648518346349537609, 648518346349539517, 648518346349537814, 648518346349539653, 648518346349537883, 648518346349539828, 648518346349537516, 648518346349537331, 648518346349537160, 648518346349537716, 648518346349536680, 648518346349537242, 648518346349537167, 648518346349534057, 648518346349536748, 648518346349528559, 648518346349527315, 648518346349537515, 648518346349532739, 648518346349532180, 648518346349539789, 648518346349537611, 648518346349537718, 648518346349539464, 648518346349539336, 648518346349527390, 648518346349539513, 648518346349536679, 648518346349524141, 648518346349537271, 648518346349535300, 648518346349537513, 648518346349539590, 648518346349531263, 648518346349537798, 648518346349536809, 648518346349528349, 648518346349538296, 648518346349537275, 648518346349539780, 648518346349532006, 648518346349535473, 648518346349539430, 648518346349537808, 648518346349537075, 648518346349538298, 648518346349525571, 648518346349537255, 648518346349537434, 648518346349537741, 648518346349539080, 648518346349537153, 648518346349534079, 648518346349538431, 648518346349537297, 648518346349537243, 648518346349537487, 648518346349531851, 648518346349491311, 648518346349537984, 648518346349523868, 648518346349531414, 648518346349538235, 648518346349538070, 648518346349536354, 648518346349535074, 648518346349507351, 648518346349537961, 648518346349537790, 648518346349538286, 648518346349532796, 648518346349537649, 648518346349533058, 648518346349531192, 648518346349536921, 648518346349539886, 648518346349492682, 648518346349538102, 648518346349539794, 648518346349537583, 648518346349539435, 648518346349538746, 648518346349524063, 648518346349537007, 648518346349537901, 648518346349540057, 648518346349539601, 648518346349538368, 648518346349531729, 648518346349539401, 648518346349539366, 648518346349530465, 648518346349537840, 648518346349539539, 648518346349538005, 648518346349538231, 648518346349539852, 648518346349531254, 648518346349527158, 648518346349537509, 648518346349534855, 648518346349536929, 648518346349536753, 648518346349533227, 648518346349534580, 648518346349537047, 648518346349533719, 648518346349537038, 648518346349537003, 648518346349537042, 648518346349539067, 648518346349496405, 648518346349521083, 648518346349538112, 648518346349538209, 648518346349539079, 648518346349539591, 648518346349537848, 648518346349537270, 648518346349537391, 648518346349539333, 648518346349532066, 648518346349537818, 648518346349540048, 648518346349536159, 648518346349537102, 648518346349538730, 648518346349539820, 648518346349539840, 648518346349540055, 648518346349539103, 648518346349537991, 648518346349519354, 648518346349539598, 648518346349532791, 648518346349538033, 648518346349532295, 648518346349539579, 648518346349538715, 648518346349534048, 648518346349537300, 648518346349536769, 648518346349533867, 648518346349537081, 648518346349539524, 648518346349503588, 648518346349539076, 648518346349537489, 648518346349535823, 648518346349537615, 648518346349537564, 648518346349539448, 648518346349537692, 648518346349537380, 648518346349531668, 648518346349539100, 648518346349539510, 648518346349539934, 648518346349537426, 648518346349538718, 648518346349524971, 648518346349538250, 648518346349525852, 648518346349539884, 648518346349537978, 648518346349537547, 648518346349539486, 648518346349534945, 648518346349530883, 648518346349539900, 648518346349538192]
# pyr cell meshes that aren't on my local drive

#[648518346349526691, 648518346349528994, 648518346349489861, 648518346349522740, 648518346349538285, 648518346349487752, 648518346349493894, 648518346349538179, 648518346349522735, 648518346349489985, 648518346349531994, 648518346349517783, 648518346349487432, 648518346349477331, 648518346349539846, 648518346349488919, 648518346349537389, 648518346349515986, 648518346349522749, 648518346349538370, 648518346349525188, 648518346349491816, 648518346349537404, 648518346349525190, 648518346349516051, 648518346349522750, 648518346349538638, 648518346349515985, 648518346349518096, 648518346349516055, 648518346349536849, 648518346349538789, 648518346349539215, 648518346349538791]
# inhibitory neurons

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        vertices = np.array(f['vertices'])
        faces = np.array(f['faces'])
    return vertices, faces

def create_pyvista_mesh(vertices, faces):
    # PyVista expects faces to be in the format (n, v1, v2, v3, ...)
    faces_with_size = np.hstack([np.full((faces.shape[0], 1), faces.shape[1]), faces])
    return pv.PolyData(vertices, faces_with_size)

def process_obj(cellid, dec_prcnt, obj_path, dec_path):
    start_time = time.time()
    
    # Create the full path to the file
    file_path = os.path.join(obj_path, str(cellid) + '.h5')
    print(f"Loading {file_path}")
    
    # Load the file
    vertices, faces = read_h5_file(file_path)
    load_time = time.time()
    print(f"Loaded {file_path} in {load_time - start_time:.2f} seconds")
    
    # Create PyVista mesh
    mesh = create_pyvista_mesh(vertices, faces)
    print(f"Original mesh info: {mesh}")
    
    # Perform decimation
    print(f"Decimating {file_path} by {dec_prcnt}%")
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt / 100)  # Reduce to x% of the original mesh
    decimate_time = time.time()
    print(f"Decimated {file_path} in {decimate_time - load_time:.2f} seconds")
    
    # Print simplified mesh information
    print(f"Simplified mesh info: {simplified_mesh}")
    
    # Ensure the output directory exists
    if not os.path.exists(dec_path):
        os.makedirs(dec_path)
    
    # Save the simplified mesh in PLY format
    output_filename = os.path.basename(file_path).replace('.h5', '.ply')
    output_path = os.path.join(dec_path, output_filename)
    
    try:
        simplified_mesh.save(output_path)
        save_time = time.time()
        print(f"Saved {output_path} in {save_time - decimate_time:.2f} seconds")
    except Exception as e:
        print(f"Error saving {output_path}: {e}")
    
    total_time = time.time() - start_time
    print(f"Total time: {total_time // 60} minutes and {total_time % 60:.2f} seconds")

# Process each cellid in the cell_list
for cellid in cell_list:
    process_obj(cellid, dec_prcnt, obj_path, dec_path)


Loading data/neuron_meshes_v185/648518346349525821.h5
Loaded data/neuron_meshes_v185/648518346349525821.h5 in 0.24 seconds
Original mesh info: PolyData (0x1dd2309e1c8)
  N Cells:	2037130
  N Points:	1018673
  N Strips:	0
  X Bounds:	2.091e+05, 4.746e+05
  Y Bounds:	1.579e+05, 3.110e+05
  Z Bounds:	8.102e+02, 8.626e+04
  N Arrays:	0

Decimating data/neuron_meshes_v185/648518346349525821.h5 by 95%
Decimated data/neuron_meshes_v185/648518346349525821.h5 in 16.91 seconds
Simplified mesh info: PolyData (0x1dd2309e408)
  N Cells:	101855
  N Points:	53892
  N Strips:	0
  X Bounds:	2.091e+05, 4.746e+05
  Y Bounds:	1.579e+05, 3.110e+05
  Z Bounds:	7.983e+02, 8.626e+04
  N Arrays:	0

Saved data/neuron_meshes_v185/dec/648518346349525821.ply in 0.09 seconds
Total time: 0.0 minutes and 17.24 seconds
